In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.0'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:14 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-02 23:49:07--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-01-02 23:49:08 (5.91 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("HW").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark.sql.functions import col,to_date,count

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
from pyspark.sql.functions import col,to_date,count
mode = "overwrite"
jdbc_url="jdbc:postgresql://database-2.cp7oam5xs6cq.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "Password#1", 
          "driver":"org.postgresql.Driver"}

url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Baby_v1_00.tsv.gz"), sep="\t", header=True)
# Show DataFrame
df.show()
df.count()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

1752932

In [6]:
review_id_table = df.select(col('review_id'),col('customer_id').cast("int"),col('product_id'),col('product_parent').cast('int'),to_date(col("review_date"),"yyyy-MM-dd").alias('review_date'))
review_id_table.show()
products_table = df.select(col('product_id'),col('product_title'))
products_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R8EWA1OFT84NX|    9970739|B00GSP5D94|     329991347| 2015-08-31|
|R2JWY4YRQD4FOP|   23538442|B00YYDDZGU|     646108902| 2015-08-31|
| RL5ESX231LZ0B|    8273344|B00BUBNZC8|     642922361| 2015-08-31|
| RRMS9ZWJ2KD08|   24557753|B00AWLZFTS|     494272733| 2015-08-31|
|R14I3ZG5E6S7YM|   46263340|B00KM60D3Q|     305813185| 2015-08-31|
|R13EPSFP5DODN5|   24557753|B00PQMRZG4|     607341708| 2015-08-31|
| R6RBP4HTE67SY|   33520065|B005DL5970|     971881542| 2015-08-31|
|R15B3EU40RSU2W|   20241560|B00C6D2WL4|      93827401| 2015-08-31|
| RP4DD53A4ZJA2|    9987983|B0083973FK|     958629336| 2015-08-31|
|R2C99DJEO4RZ4K|   52570308|B00RLYG2S2|     147324304| 2015-08-31|
| REV51EW323H8W|    9287389|B010UX9T5I|     446691106| 2015-08-31|
|R2GQ3W03WIUZKE|   32840762|B00VWBY7SC|     271204734| 2015-08

In [7]:
products_table = df.select(col('product_id'),col('product_title')).distinct()
products_table.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00Y8QMQN6|Best Baby Teether...|
|B002WN2BY6|The First Years T...|
|B000AV7O4O|Boon Frog Pod Bat...|
|B00UFOBDNY|Kinsa Smart Stick...|
|B00XF7DZ34|Critter Piller Ki...|
|B00WKIR22E|Poly Comfortable ...|
|B001EU35BQ|Peg Perego Tatami...|
|B001TWI81G|Thermos Reusable ...|
|B007EJ7J4U|Kaloo Doudou Beig...|
|B00M8MGGQA|Large Baby Bag Or...|
|B00Y2CJWKQ|Clinical Digital ...|
|B00Y8ION80|★ Baby Pacifier &...|
|B00UJOWZ96|Corner Cushion Ba...|
|B00DZ3K64Y|Sassy Soft Grip N...|
|B00JBYPKE2|Maxi Cosi Pria 85...|
|B00HK6GWTU|Ju-Ju-Be Be Light...|
|B00IYU4O8W|   Lamaze Cloth Book|
|B00SCDM4FQ|Olababy Silicone ...|
|B004UOTWJ2|Charlie Banana - ...|
|B001LF3YQK|Skip Hop Baby Swi...|
+----------+--------------------+
only showing top 20 rows



In [8]:
customers_table = df.select(col('customer_id').cast('int')).groupBy('customer_id').agg(count('customer_id').alias("customer_count") )
customers_table.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   23042837|             2|
|   44524872|             1|
|   16711087|             1|
|   14116630|             3|
|   10548283|             1|
|   12879980|            15|
|   11913481|             1|
|   49875798|             3|
|   40993003|             1|
|    6272214|             3|
|     417296|             1|
|   36524024|             2|
|   16146148|             1|
|   43274367|             1|
|   38692392|            14|
|   24359722|             1|
|   45450058|             5|
|    1960118|             1|
|   36980552|             1|
|   10200363|             2|
+-----------+--------------+
only showing top 20 rows



In [9]:
vine_table =df.select(col('review_id'),col('star_rating').cast("int"),col('helpful_votes').cast("int"),col('total_votes').cast('int'),col("vine"))
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R8EWA1OFT84NX|          5|            0|          0|   N|
|R2JWY4YRQD4FOP|          5|            0|          0|   N|
| RL5ESX231LZ0B|          5|            0|          0|   N|
| RRMS9ZWJ2KD08|          5|            0|          0|   N|
|R14I3ZG5E6S7YM|          5|            0|          0|   N|
|R13EPSFP5DODN5|          4|            0|          0|   N|
| R6RBP4HTE67SY|          5|            0|          0|   N|
|R15B3EU40RSU2W|          5|            0|          0|   N|
| RP4DD53A4ZJA2|          5|            0|          0|   N|
|R2C99DJEO4RZ4K|          5|            3|          4|   N|
| REV51EW323H8W|          5|            0|          0|   N|
|R2GQ3W03WIUZKE|          5|            0|          0|   N|
| RTI1YI7K6GE3D|          5|            0|          0|   N|
|R3V9C2C0SPSZU6|          5|            

In [10]:

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [11]:
products_table.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [12]:
customers_table.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [13]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)